In [31]:
import numpy as np 
import math

x = [0.298, 0.303, 0.310, 0.317, 0.323, 0.330, 0.339]
y = [3.2557, 3.174, 3.1218, 3.0482, 2.9875, 2.9195, 2.8359]
#x1 = 0.314
#x2 = 0.253
#x3 = 0.512
#x4 = 0.337

# Интерполяционный многочлен Лагранжа
Вариант 5

In [30]:
def interpolate_lagrange(x1) :
    res = 0.0
    for i in range (0, len(x)):
        p = 1.0
        for j in range (0, len(x)):
            if(i != j):
                p *= (x1 - x[j])/(x[i]-x[j])
        res += p*y[i]
    return res

# Интерполяционный многочлен Ньютона с разделенными разностями

In [78]:
def f(ind):
    if len(ind) < 2:
        return None
    if len(ind) == 2:
        return (y[ind[1]] - y[ind[0]])/(x[ind[1]] - x[ind[0]])
    arr1 = range(1, len(ind))
    arr2 = range(0, len(ind) - 1)
    print (ind)
    print ( (f(arr1) - f(arr2))/(x[ind[len(ind) - 1]] - x[ind[0]]))
    print()
    return (f(arr1) - f(arr2))/(x[ind[len(ind) - 1]] - x[ind[0]])

def interpolate_newthon(x1):
    res = y[0]
    arr = range(1, 3)
    for i in range (1, len(x) - 1):
        s = f(arr)
        p = 1.0
        for j in range (0, i-1):
            p *= (x1 - x[j])
        res += s * p
    return res
        
print(interpolate_newthon(0.22))

12.870569230769231


In [62]:
x = [0.15, 0.2, 0.33, 0.77, 0.62]
y = [1.25, 2.38, 3.79, 5.44, 7.14]